In [1]:
import pandas as pd
import gensim.models
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')

[nltk_data] Downloading package punkt to /Users/2010y/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/2010y/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
folder='nashville_union_american'

In [3]:
import spacy
nlp=spacy.load('en_core_web_sm')

/Users/2010y/opt/miniconda3/envs/viraltext/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
os.chdir('/Users/2010y/opt/viraltext/snippet-fasttext')

In [6]:
# df_negro=pd.read_csv('two_extended_anti_slavery_bugle_negro.csv')
df_slave=pd.read_csv('two_extended_'+folder+'_slave'+'.csv')
df_servant=pd.read_csv('two_extended_'+folder+'_servant'+'.csv')

In [7]:
frame=[df_slave, df_servant]

pre-processing

In [8]:
df=pd.concat(frame)

In [9]:
def lemmatization(dataframe:pd.DataFrame()):
    dataframe['stopword']=dataframe['context'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
    dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]', '')
    dataframe['lower']=dataframe['punct'].str.lower()
    dataframe['lemma']=dataframe['lower'].apply(lambda row:' '.join([w.lemma_ for w in nlp(row)]))
    dataframe['token']=dataframe['lemma'].apply(word_tokenize)
    return dataframe

In [10]:
%%time
df=lemmatization(df)

/var/folders/bs/0dqdld413qjd4wfpkl2zqd3w0000gn/T/ipykernel_18220/2545040651.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]', '')


CPU times: user 4min 10s, sys: 604 ms, total: 4min 10s
Wall time: 4min 10s


word2vec

In [189]:
from gensim.models.word2vec import Word2Vec

In [190]:
model=Word2Vec([row for row in df['token']], min_count=10, workers=3, window=5, sg=1)

In [191]:
from gensim.models import KeyedVectors
servant_slave=model.wv.most_similar(positive=['servant'], negative=['slave'], topn=100)
slave_servant=model.wv.most_similar(positive=['slave'], negative=['servant'], topn=100)

In [192]:
dataframe=pd.DataFrame(slave_servant, columns=('token', 'similarity'))
outname='slave_servant_'+folder+'_min10.csv'
outdir='/Users/2010y/opt/viraltext/word2vec'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname=os.path.join(outdir, outname)
dataframe.to_csv(fullname)

In [193]:
dataframe=pd.DataFrame(servant_slave, columns=('token', 'similarity'))
outname='servant_slave_'+folder+'_min10.csv'
outdir='/Users/2010y/opt/viraltext/word2vec'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname=os.path.join(outdir, outname)
dataframe.to_csv(fullname)